In [3]:
%cd ~/NetBeansProjects/ExpLosion/
from notebooks.common_imports import *
from gui.output_utils import *

def get_exp_ids_for_varying_k(corpus='amazon_grouped-tagged'):
    # curve when varying k
    query_dict = {'expansions__allow_overlap': 0,
                  'expansions__decode_handler': 'SignifiedOnlyFeatureHandler',
                 'document_features_ev': 'AN+NN',
                 'document_features_tr': 'J+N+AN+NN',
                 'expansions__k__in': [1, 3, 5, 10, 20, 30, 40, 50, 75, 100],
                 'labelled': 'amazon_grouped-tagged',
                 'expansions__neighbour_strategy': 'linear',
                 'expansions__noise': 0.0,
                 'expansions__use_random_neighbours': 0,
                 'expansions__use_similarity': 0,
                'expansions__vectors__composer__in': ['Add', 'Mult', 'Left', 'Right'], # , todo Verb???
                'expansions__vectors__algorithm': 'word2vec',
                 'expansions__vectors__dimensionality': 100,
                 'expansions__vectors__rep': 0,
                 'expansions__vectors__unlabelled': 'wiki',
                 'expansions__vectors__unlabelled_percentage': 100,                 
}
    return Experiment.objects.filter(**query_dict)#.values_list('id', flat=True)

/Users/miroslavbatchkarov/NetBeansProjects/ExpLosion


In [6]:
experiments = get_exp_ids_for_varying_k()
ids = [e.id for e in experiments if e.expansions.entries_of is None]
print('ids are', ids)
print(ids[19])

df = dataframe_from_exp_ids(ids, {'k':'expansions__k', 
                                  'Composer':'expansions__vectors__composer'}).convert_objects(convert_numeric=True)
grid = sns.factorplot(x='k', y='Accuracy', data=df, hue='Composer',
                      kind='point');
plt.savefig('plot-w2v_effect-of-k.pdf', format='pdf', dpi=300)

ids are [34, 35, 36, 37, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
62


DoesNotExist: Results matching query does not exist.

In [ ]:
def f1(x):
    return '%1.2f' % x

ddf = df.drop('folds', axis=1).groupby(['Composer', 'k']).agg([np.mean, np.std])
ddf.columns = ddf.columns.droplevel(0)#.reset_index()
ddf['Accuracy'] = ddf['mean'].map(f1) + "$\pm$" + ddf['std'].map(f1)
ddf = ddf.drop(['mean', 'std'], axis=1).reset_index()
print(ddf.pivot_table(values='Accuracy', index='k', 
                      columns='Composer', aggfunc=lambda x: x).to_latex(escape=False))
ddf = df.groupby(['Composer', 'k']).agg('mean').reset_index() # no need to drop unwanted columns
print(ddf.pivot_table(values='Accuracy', index='k', columns='Composer').to_latex(float_format=f1))

In [7]:
set(range(1, 267)).difference(set(Results.objects.values_list('id', flat=True)))

{62, 247}